# Install the required libraries

In [1]:
!pip install google-generativeai --quiet

# Configure the LLM for our use case

Get a free Gemini API key using the following wesbite: https://aistudio.google.com/app/apikey and paste it in a txt file called `google_api_path.txt`.

In [2]:
import os
import getpass
google_api_path = 'google_api_path.txt'
if "GEMINI_API_KEY" not in os.environ:
    try:
        with open(google_api_path, "r") as file:
            api_key = file.read().strip()
            os.environ['GEMINI_API_KEY'] = api_key
    except:
        os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Using LLM only

In [3]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [5]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config=generation_config,
)

In [6]:
chat_session = model.start_chat()

In [7]:
response = chat_session.send_message("Who is the president of the United States of America?")

In [8]:
print(response.text)

Joe Biden



# Using a LLM and a tool

In this case, our tool would be google search

In [9]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config=generation_config,
  tools = [
    genai.protos.Tool(
      google_search_retrieval = genai.protos.GoogleSearchRetrieval(
        dynamic_retrieval_config = genai.protos.DynamicRetrievalConfig(
          mode = genai.protos.DynamicRetrievalConfig.Mode.MODE_DYNAMIC,
          dynamic_threshold = 0.3,
        ),
      ),
    ),
  ],
)
chat_session = model.start_chat()

In [10]:
response = chat_session.send_message("Who is the president of United States of America?")

In [11]:
print(response.text)

Donald J. Trump is the president of the United States.



In [12]:
response.candidates[0].grounding_metadata.grounding_chunks

[web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYsm-LoWw7XlsyiSz2dq-i2csYcMEd_x5mAk3otR_LVfVJp_AsuDZmHK0Mbu5_ITFYDyfMkLaNfQdHvDAHgPmM-qJfCbh2OGCMrWO4rcNlyEDEIxK-zdJxDwJesJPGYcGOqUUJJj4MT2QOWu0h_Mo0WwNC8sMvjtAh2oFg=="
  title: "usembassy.gov"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYsHn0DFAnSDRJlym2WANLvNmfqYW4B4o8G-yym8uwjenZ97BGfR1oBlKt5wre6O4AxWOB6zjYuyenTIDDfomftNZSZxClCBvUUI9oEBDvHsNv0UKvg_Tyvebg=="
  title: "usa.gov"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYuinkf2w7lBByCTCC72pgeF6VZFK5QicAxLN4oXah9lQv5-V4tkUeYNlfh-lCGYOwWGlHPXusxjpGw4Iu9mroFlTJxyaqiFK6mqhvlx1pwJFd9efoaNtOGR1kWzC3O2eRZmjQpLLyQZyBG990GowAPLmU-RpLlMHd6Xd9nz9kvfUyBh9EnhbOyjwBMbGbX9A12GlIxHKWqqoJVtijhxXu2bonaM"
  title: "weforum.org"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYvlsEwrxSUzuYbVIN2WRmFFP4gOfQ9wQdPZIZgWrbpnaX6MYDK6Q-_3G6bdUyWz1uYpm6FfutA9DA

# Using LLM, prompt engineering and tooling

In [13]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config=generation_config,
  system_instruction="You are a customer service agent for ServiceNow. Only answer questions about Customer Service and ServiceNow.",
)

chat_session = model.start_chat(
  history=[
  ]
)

In [14]:
response = chat_session.send_message("Who is the president of the united states")

In [15]:
print(response.text)

I do not have access to that information.  My knowledge is limited to customer service and ServiceNow.



In [16]:
response = chat_session.send_message("How do I create an incident in ServiceNow?")

In [17]:
print(response.text)

There are several ways to create an incident in ServiceNow, depending on your access and the specific ServiceNow instance you're using.  Generally, you'll need to:

1. **Navigate to the Incident form:**  This can be done through the ServiceNow navigation menu, often by searching for "Incident" or using a shortcut link.  Different instances may have different layouts.

2. **Fill in the required fields:**  These fields will vary, but typically include:
    * **Short Description:**  A concise summary of the problem.
    * **Category:**  Classifies the issue (e.g., Application, Hardware, Network).
    * **Priority:**  Indicates the urgency of the issue (e.g., Low, Medium, High, Urgent).
    * **Assignment Group:**  If you know the team responsible.
    * **Description:**  A more detailed explanation of the problem, including steps to reproduce, any error messages, and relevant information.
    * **Impact/Urgency/Urgency (These might be grouped as a single field):** A measure of the problem